<a href="https://colab.research.google.com/github/rodrigorenemenegazzo/rodrigorenemenegazzo/blob/main/Text_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Text Mining**

"Text mining (also known as text analysis), is the process of transforming unstructured text into structured data for easy analysis. Text mining uses natural language processing (NLP), allowing machines to understand the human language and process it automatically." [Source](https://monkeylearn.com/text-mining/#:~:text=Text%20mining%20(also%20known%20as,language%20and%20process%20it%20automatically.)